In [1]:
import pandas as pd
import numpy as np
import re

f = open("data/data19.txt", "r")
lines = [line[:-1] for line in f.readlines()]

lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]

def get_costs(line):
    d = {}
    k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = tuple([int(z) for z in re.findall(r'\d+', line)])
    return k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs

print(len(lines))
print(get_costs(lines[0]))

2
(1, 4, 2, 3, 14, 2, 7)


In [2]:
k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(lines[0])
print(k, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs)

def get_maxperf(t, ore, clay, obs, geo, n_ore, n_clay, n_obs, n_geo):

    if t==0:
        return geo#, []
    
    res = []
    #strat = []
    ore2 = ore + n_ore
    clay2 = clay + n_clay
    obs2 = obs + n_obs
    geo2 = geo + n_geo    

    #r, s0 = get_maxperf(t-1, ore2, clay2, obs2, geo2, n_ore, n_clay, n_obs, n_geo)
    res.append(get_maxperf(t-1, ore2, clay2, obs2, geo2, n_ore, n_clay, n_obs, n_geo))
    #strat.append(["wait"] + s0)
    if ore >= c_ore_ore:
        #r, s1 = get_maxperf(t-1, ore2 - c_ore_ore, clay2, obs2, geo2, n_ore + 1, n_clay, n_obs, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_ore_ore, clay2, obs2, geo2, n_ore + 1, n_clay, n_obs, n_geo))
        #strat.append(["r_ore"] + s1)
    if ore >= c_clay_ore:
        #r, s2 = get_maxperf(t-1, ore2 - c_clay_ore, clay2, obs2, geo2, n_ore, n_clay + 1, n_obs, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_clay_ore, clay2, obs2, geo2, n_ore, n_clay + 1, n_obs, n_geo))
        #strat.append(["r_clay"] + s2)
    if (ore >= c_obs_ore) and (clay >= c_obs_clay):
        #r, s3 = get_maxperf(t-1, ore2 - c_obs_ore, clay2 - c_obs_clay, obs2, geo2, n_ore, n_clay, n_obs + 1, n_geo)
        res.append(get_maxperf(t-1, ore2 - c_obs_ore, clay2 - c_obs_clay, obs2, geo2, n_ore, n_clay, n_obs + 1, n_geo))
        #strat.append(["r_obs"] + s3)
    if (ore >= c_geo_ore) and (obs >= c_geo_obs):
        #r, s4 = get_maxperf(t-1, ore2 - c_geo_ore, clay2 , obs2 - c_geo_obs, geo2, n_ore, n_clay, n_obs, n_geo + 1)
        res.append(get_maxperf(t-1, ore2 - c_geo_ore, clay2 , obs2 - c_geo_obs, geo2, n_ore, n_clay, n_obs, n_geo + 1))
        #strat.append(["r_geo"] + s4)
    
    m = max(res)
    #imax = res.index(max(res))

    return m#, strat[imax]

print(get_maxperf(10, 0, 0, 0, 0, 1, 0, 0, 0))


1 4 2 3 14 2 7
0


In [3]:
get_maxperf(4, 4, 25, 7, 2, 1, 4, 2, 1)


9

In [4]:
get_maxperf(14, 4, 15, 0, 0, 1, 3, 0, 0)


9

In [16]:
lines = [
    "Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.",
    "Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."
]


def update(x):
    y = x.copy()
    y[:4] += y[4:]
    return y

L_states = [np.array([0, 0, 0, 0, 1, 0, 0, 0], dtype='int8')]

tmax = 24
def get_options(line, tmax, verbose=True):
    _, c_ore_ore, c_clay_ore, c_obs_ore, c_obs_clay, c_geo_ore, c_geo_obs = get_costs(line)
    r_ore = np.array([- c_ore_ore, 0, 0, 0, 1, 0, 0, 0], dtype='int8')
    r_clay = np.array([- c_clay_ore, 0, 0, 0, 0, 1, 0, 0], dtype='int8')
    r_obs = np.array([- c_obs_ore, - c_obs_clay, 0, 0, 0, 0, 1, 0], dtype='int8')
    r_geo = np.array([-c_geo_ore, 0, - c_geo_obs, 0, 0, 0, 0, 1], dtype='int8')
    L_states = [np.array([0, 0, 0, 0, 1, 0, 0, 0], dtype='int8')]
    maxore = max([c_ore_ore, c_clay_ore, c_obs_ore, c_geo_ore])

    for t in range(1, tmax+1):
        list_options = []
        for x in L_states:
            y = update(x)
            n_choices = 0
            if (x[0] >= c_geo_ore) and (x[2] >= c_geo_obs):
                list_options.append(y+r_geo)
                n_choices += 2
            if (n_choices<2) and (x[0] >= c_obs_ore) and (x[1] >= c_obs_clay):
                list_options.append(y+r_obs)
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_clay_ore):
                list_options.append(y+r_clay)
                n_choices += 1
            if (n_choices<2) and (x[0] >= c_ore_ore):
                list_options.append(y+r_ore)
                n_choices += 0
            if n_choices<2:
                list_options.append(y)
        m = max([l[3] for l in list_options])
        L_1 = [l for l in list_options if l[3]>m-2]
        L_states = [l for l in L_1 if l[0]<2*maxore]
        if verbose:
            print(t, len(L_1), len(L_states))
    return L_states

l1 = get_options(lines[0], tmax)
#l2 = get_options(lines[1], tmax)
max([s[3] for s in l1])#, max([s[3] for s in l2])

1 1 1
2 1 1
3 2 2
4 3 3
5 7 7
6 12 12
7 29 29
8 57 56
9 140 138
10 305 288
11 743 693
12 1656 1447
13 3601 3027
14 7148 5789
15 12897 10531
16 24503 19796
17 44750 36063
18 78780 65048
19 140126 116153
20 2709 2599
21 5210 4987
22 392 392
23 547 535
24 773 748


9

In [17]:
f = open("data/data19.txt", "r")
lines = [line[:-1] for line in f.readlines()]
tmax = 24

best_k = {}
for i, line in enumerate(lines):
    options = get_options(line, 24, False)
    best_k[i+1] = max([s[3] for s in options])
    print(i+1, best_k[i+1])
s = 0
for i in best_k.keys():
    s += i*best_k[i]
print(s) # 1273 is too low

1 3
2 13
3 0
4 0
5 0
6 8
7 0
8 6
9 0
10 2
11 0
12 0
13 1
14 12
15 0
16 0
17 0
18 0
19 0
20 0
21 6
22 1
23 0
24 15
25 1
26 3
27 7
28 1
29 1
30 3
1273
